# Exploring inference methods to plan inference pipeline

In [1]:
import hopsworks
from datetime import datetime
import pandas as pd

In [2]:
# Today's date ----- is this needed/helpful anywhere?
# today = pd.to_datetime('2023-12-13').date()
today = datetime.now().strftime('%Y-%m-%d')
#today = datetime.now().date()
print(type(today))
print(today)

<class 'str'>
2023-12-25


## Get stored news articles

In [3]:
project = hopsworks.login()
fs = project.get_feature_store()

news_fg = fs.get_feature_group(name="news_articles", version=6)
news_df = news_fg.read()
news_df.head()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/187540
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using ArrowFlight (7.21s) from Hopsworks, using ArrowFlight.   Reading data from Hopsworks, using ArrowFlight...   Reading data from Hopsworks, using ArrowFlight.   Reading data from Hopsworks, using ArrowFlight...   Reading data from Hopsworks, using ArrowFlight.   Reading data from Hopsworks, using ArrowFlight...   


,article_id,title,link,description,content,pubdate,source_id,country,category,language
0,dcc9522620008f6ef97502df973dd50f,CRA has fired 185 employees for ‘inappropriate...,https://lethbridgenewsnow.com/2023/12/20/cra-h...,OTTAWA - The Canada Revenue Agency says 185 em...,OTTAWA — The Canada Revenue Agency says 185 em...,2023-12-20,lethbridgenewsnow,['canada'],['top'],english
1,e8ed1747c671fe67b9719c42338e433d,How to build the ultimate iPhone 15 Pro 4K60 P...,https://appleinsider.com/inside/iphone-15-pro/...,Shooting 4K60 ProRes video on an iPhone 15 Pro...,Affiliate Disclosure If you buy through our li...,2023-12-20,appleinsider,"['united kingdom', 'australia', 'india', 'unit...",['technology'],english
2,a37e8ab250f2c020f0a44d044ada7724,Scientists uncover secrets of brain developmen...,https://medicalxpress.com/news/2023-12-scienti...,The human brain continues to be built after we...,The human brain continues to be built after we...,2023-12-20,medicalxpress,['canada'],['top'],english
3,629ebaba27eeb1a4d2bca36ee79f2bd9,Glenties to get a new pedestrian crossing,https://www.donegaldaily.com/2023/12/20/glenti...,Road-users and pedestrians in Glenties have be...,Road-users and pedestrians in Glenties have be...,2023-12-20,donegaldaily,['ireland'],['top'],english
4,8e7ee096666a027faec3eeead571bcef,Pokémon Scarlet and Violet’s DLC expansion is ...,https://www.techradar.com/gaming/nintendo-swit...,An epilogue for Pokémon Scarlet and Violet’s H...,Just when you thought that Pokémon Scarlet and...,2023-12-20,techradar,"['australia', 'india']",['technology'],english


In [4]:
type(news_df["country"].iloc[0])

str

In [5]:
from ast import literal_eval

#news_df["country"] = news_df["country"].apply(literal_eval)
news_df["country"].iloc[0]


"['canada']"

In [6]:
print("Number of articles:", len(news_df))

Number of articles: 269


In [7]:
news_df['pubdate'].dtype

dtype('O')


## Only keep today's articles

In [8]:
news_df = news_df[news_df['pubdate'] == today]
print("Number of articles:", len(news_df))

Number of articles: 39


## Add sentiments for articles

In [9]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
PydanticDeprecatedS

In [10]:
def format_sentiment(sentiment):
    if sentiment['label'] == 'NEGATIVE':
        return sentiment['score'] * -1
    else:
        return sentiment['score']

def get_sentiment_value(news_object, sentiment_pipeline):
    return format_sentiment(sentiment_pipeline(news_object['title'])[0])

In [11]:
news_df['sentiment'] = news_df.apply(get_sentiment_value, sentiment_pipeline=sentiment_pipeline, axis=1)

In [12]:
news_df['sentiment'] 

230   -0.710903
231   -0.993367
232    0.997036
233    0.997036
234   -0.647142
235   -0.568736
236   -0.991280
237   -0.993367
238   -0.988592
239    0.820120
240    0.997656
241   -0.996427
242   -0.977887
243   -0.931647
244    0.992721
245    0.997656
246   -0.968012
247    0.994380
248   -0.998624
249    0.998703
250   -0.997069
251   -0.993995
252   -0.961167
253    0.998866
254   -0.947101
255    0.996126
256   -0.997069
257   -0.999413
258    0.995404
259    0.999520
260    0.997036
261   -0.931647
262    0.990994
263   -0.990683
264   -0.993367
265   -0.999344
266    0.987263
267   -0.993367
268   -0.899623
Name: sentiment, dtype: float64

In [13]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39 entries, 230 to 268
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   article_id   39 non-null     object 
 1   title        39 non-null     object 
 2   link         39 non-null     object 
 3   description  39 non-null     object 
 4   content      39 non-null     object 
 5   pubdate      39 non-null     object 
 6   source_id    39 non-null     object 
 7   country      39 non-null     object 
 8   category     39 non-null     object 
 9   language     39 non-null     object 
 10  sentiment    39 non-null     float64
dtypes: float64(1), object(10)
memory usage: 3.7+ KB


## Calculate today's average sentiment

In [14]:
avg_sentiment = news_df['sentiment'].mean()
avg_sentiment

-0.19767473752682024

## Find today's most positive article

In [15]:
most_positive = news_df.loc[news_df['sentiment'].idxmax()]
most_positive["country"]

"['canada']"

In [16]:
most_positive = pd.DataFrame(most_positive).T
most_positive.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 259 to 259
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   article_id   1 non-null      object
 1   title        1 non-null      object
 2   link         1 non-null      object
 3   description  1 non-null      object
 4   content      1 non-null      object
 5   pubdate      1 non-null      object
 6   source_id    1 non-null      object
 7   country      1 non-null      object
 8   category     1 non-null      object
 9   language     1 non-null      object
 10  sentiment    1 non-null      object
dtypes: object(11)
memory usage: 96.0+ bytes


In [17]:
most_positive.pubdate

259    2023-12-25
Name: pubdate, dtype: object